In [3]:

#Task 2; ETL
import os
import pandas as pd

# Define directory
OUTPUT_DIR = r"C:\Users\USER\OneDrive\Desktop\Data Warehousing\DSA204 final Exam"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Define file paths
INPUT_CSV = os.path.join(OUTPUT_DIR, "Data", "Amazon Sale Report.csv")
OUTPUT_CSV = os.path.join(OUTPUT_DIR, "Processed_Amazon_Sale_Report.csv")

# Load the dataset
df = pd.read_csv(INPUT_CSV)

# Display first 5 rows
df.head()


C:\Users\USER\AppData\Local\Temp\ipykernel_22244\3789260593.py:14: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_CSV)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [5]:
# -------------------------------
# STEP 1: EXTRACTION & FULL CLEANING
# -------------------------------

import pandas as pd
import numpy as np

print("Rows before cleaning:", len(df))

# 1. Remove useless unnamed columns
df = df.loc[:, ~df.columns.str.contains("Unnamed")]

# 2. Strip whitespace from column names
df.columns = df.columns.str.strip()

# 3. Convert Date column to datetime
df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y", errors="coerce")


# 4. Identify numeric columns and convert
numeric_cols = ["Qty", "Amount", "ship-postal-code"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill numeric NaN with 0
df[numeric_cols] = df[numeric_cols].fillna(0)

# 5. Clean boolean-like columns (B2B)
if "B2B" in df.columns:
    df["B2B"] = df["B2B"].fillna(False)
    df["B2B"] = df["B2B"].astype(bool)

# 6. Clean fulfilled-by: replace NaN with "Unknown"
if "fulfilled-by" in df.columns:
    df["fulfilled-by"] = df["fulfilled-by"].fillna("Unknown")

# 7. Clean currency column
if "currency" in df.columns:
    df["currency"] = df["currency"].fillna("Unknown")
    df["currency"] = df["currency"].astype(str).str.upper()

# 8. Standardize text fields (string cleaning)
text_columns = df.select_dtypes(include="object").columns.tolist()

for col in text_columns:
    df[col] = df[col].astype(str).str.strip().str.title().replace("Nan", "Unknown")

# 9. Drop rows with missing or invalid dates
df = df.dropna(subset=["Date"])

# Display cleaned data
print("Rows after cleaning:", len(df))
print("\nCLEANED DATA INFO:")
print(df.info())

df.head()


Rows before cleaning: 128975
Rows after cleaning: 128975

CLEANED DATA INFO:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               128975 non-null  int64         
 1   Order ID            128975 non-null  object        
 2   Date                128975 non-null  datetime64[ns]
 3   Status              128975 non-null  object        
 4   Fulfilment          128975 non-null  object        
 5   Sales Channel       128975 non-null  object        
 6   ship-service-level  128975 non-null  object        
 7   Style               128975 non-null  object        
 8   SKU                 128975 non-null  object        
 9   Category            128975 non-null  object        
 10  Size                128975 non-null  object        
 11  ASIN                128975 non-null  object        
 12  Courier S

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.In,Standard,Set389,Set389-Kr-Np-S,Set,...,0,Inr,647.62,Mumbai,Maharashtra,400081.0,In,Unknown,False,Easy Ship
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered To Buyer,Merchant,Amazon.In,Standard,Jne3781,Jne3781-Kr-Xxxl,Kurta,...,1,Inr,406.00,Bengaluru,Karnataka,560085.0,In,Amazon Plcc Free-Financing Universal Merchant ...,False,Easy Ship
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.In,Expedited,Jne3371,Jne3371-Kr-Xl,Kurta,...,1,Inr,329.00,Navi Mumbai,Maharashtra,410210.0,In,In Core Free Shipping 2015/04/08 23-48-5-108,True,Unknown
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.In,Standard,J0341,J0341-Dr-L,Western Dress,...,0,Inr,753.33,Puducherry,Puducherry,605008.0,In,Unknown,False,Easy Ship
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.In,Expedited,Jne3671,Jne3671-Tu-Xxxl,Top,...,1,Inr,574.00,Chennai,Tamil Nadu,600073.0,In,Unknown,False,Unknown


In [16]:
# ---------------------------------------------
# STEP 2: TRANSFORMATION
# ---------------------------------------------
import numpy as np

print("\n--- TRANSFORMATION STARTED ---")

# Make a safe copy to avoid warnings
df = df.copy()

# ----------------------------------------------------
# 1. Convert Date column
# ----------------------------------------------------
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Remove rows with invalid dates
df = df.dropna(subset=["Date"])

# ----------------------------------------------------
# 2. Create new useful columns
# ----------------------------------------------------

# Total sales for each order line
df["TotalSales"] = df["Qty"] * df["Amount"]

# Extract time-based features
df["OrderMonth"] = df["Date"].dt.to_period("M").astype(str)
df["OrderYear"] = df["Date"].dt.year

# Convert B2B Yes/No to 1/0
df["B2B_Flag"] = df["B2B"].apply(
    lambda x: 1 if str(x).strip().lower() == "yes" else 0
)

# Create region grouping
df["ShippingRegion"] = df["ship-country"].apply(
    lambda x: "Local (Kenya)" if str(x).strip().lower() == "kenya" else "International"
)

# Categorize order value
df["ValueCategory"] = pd.cut(
    df["TotalSales"],
    bins=[0, 1000, 5000, np.inf],
    labels=["Low", "Medium", "High"]
)

# ----------------------------------------------------
# 3. Remove outliers
# ----------------------------------------------------
before_cleaning = len(df)
df = df[(df["Qty"] >= 0) & (df["Amount"] > 0)]
after_cleaning = len(df)

print(f"Outliers removed: {before_cleaning - after_cleaning}")

# ----------------------------------------------------
# 4. Filter for the last 3 months (Dynamic)
# ----------------------------------------------------
dataset_max_date = df["Date"].max()
three_months_ago = dataset_max_date - pd.DateOffset(months=3)

df_last_3_months = df[df["Date"].between(three_months_ago, dataset_max_date)]

print(f"Dataset Max Date: {dataset_max_date}")
print(f"Filtering from: {three_months_ago}")
print(f"Rows in last 3 months: {len(df_last_3_months)}")

# ----------------------------------------------------
# 5. CUSTOMER DIMENSION
# ----------------------------------------------------
customer_dim = (
    df_last_3_months.groupby(["ship-postal-code", "ship-city", "ship-country"])
    .agg(
        TotalOrders=("Order ID", "count"),
        TotalQuantity=("Qty", "sum"),
        TotalSpent=("TotalSales", "sum")
    )
    .reset_index()
)

# Add surrogate CustomerID
customer_dim.insert(0, "CustomerID", range(1, len(customer_dim) + 1))

print("\nCustomer Dimension Created:", len(customer_dim))

print("\n--- TRANSFORMATION COMPLETE ---")

# Show sample of transformed data
df_last_3_months.head()



--- TRANSFORMATION STARTED ---
Outliers removed: 0
Dataset Max Date: 2022-06-29 00:00:00
Filtering from: 2022-03-29 00:00:00
Rows in last 3 months: 118837

Customer Dimension Created: 14024

--- TRANSFORMATION COMPLETE ---


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,ship-country,promotion-ids,B2B,fulfilled-by,TotalSales,OrderMonth,OrderYear,B2B_Flag,ShippingRegion,ValueCategory
0,0,405-8078784-5731545,2022-04-30,Cancelled,Merchant,Amazon.In,Standard,Set389,Set389-Kr-Np-S,Set,...,In,Unknown,False,Easy Ship,0.0,2022-04,2022,0,International,NaN
1,1,171-9198151-1101146,2022-04-30,Shipped - Delivered To Buyer,Merchant,Amazon.In,Standard,Jne3781,Jne3781-Kr-Xxxl,Kurta,...,In,Amazon Plcc Free-Financing Universal Merchant ...,False,Easy Ship,406.0,2022-04,2022,0,International,Low
2,2,404-0687676-7273146,2022-04-30,Shipped,Amazon,Amazon.In,Expedited,Jne3371,Jne3371-Kr-Xl,Kurta,...,In,In Core Free Shipping 2015/04/08 23-48-5-108,True,Unknown,329.0,2022-04,2022,0,International,Low
3,3,403-9615377-8133951,2022-04-30,Cancelled,Merchant,Amazon.In,Standard,J0341,J0341-Dr-L,Western Dress,...,In,Unknown,False,Easy Ship,0.0,2022-04,2022,0,International,NaN
4,4,407-1069790-7240320,2022-04-30,Shipped,Amazon,Amazon.In,Expedited,Jne3671,Jne3671-Tu-Xxxl,Top,...,In,Unknown,False,Unknown,574.0,2022-04,2022,0,International,Low


In [9]:
# ------------------------------------------------------------
# STEP 3: LOAD INTO SQLITE DATA WAREHOUSE
# ------------------------------------------------------------
import sqlite3

# Database output path
DB_PATH = os.path.join(OUTPUT_DIR, "retail_dw.db")

# Connect to SQLite
conn = sqlite3.connect(DB_PATH)

print("\n--- LOADING INTO SQLITE DATA WAREHOUSE ---")

# -----------------------------------
# 1. LOAD CUSTOMER DIMENSION
# -----------------------------------
customer_dim.to_sql("CustomerDim", conn, if_exists="replace", index=False)

print("CustomerDim loaded:", len(customer_dim))

# -----------------------------------
# 2. CREATE TIME DIMENSION
# -----------------------------------

# Extract unique dates
time_dim = df_last_year[["Date"]].drop_duplicates().copy()
time_dim["Date"] = pd.to_datetime(time_dim["Date"], errors="coerce")

# Add useful time columns
time_dim["Day"] = time_dim["Date"].dt.day
time_dim["Month"] = time_dim["Date"].dt.month
time_dim["MonthName"] = time_dim["Date"].dt.month_name()
time_dim["Quarter"] = time_dim["Date"].dt.quarter
time_dim["Year"] = time_dim["Date"].dt.year

# Add surrogate key
time_dim.insert(0, "TimeID", range(1, len(time_dim) + 1))

# Load into DB
time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)

print("TimeDim loaded:", len(time_dim))

# -----------------------------------
# 3. CREATE SALES FACT TABLE
# -----------------------------------

# STEP: merge fact table with CustomerDim to attach CustomerID
sales_fact = df_last_year.merge(
    customer_dim,
    on=["ship-postal-code", "ship-city", "ship-country"],
    how="left"
)

# Attach TimeID
sales_fact = sales_fact.merge(time_dim[["TimeID", "Date"]], on="Date", how="left")

# Select FACT TABLE columns only
sales_fact = sales_fact[[
    "Order ID",
    "CustomerID",
    "TimeID",
    "SKU",
    "Category",
    "Qty",
    "Amount",
    "TotalSales",
    "Status",
    "Sales Channel",
    "ship-city",
    "ship-state",
    "ship-country",
    "ShippingRegion",
    "ValueCategory",
    "B2B_Flag"
]]

# Load into SQLite
sales_fact.to_sql("SalesFact", conn, if_exists="replace", index=False)

print("SalesFact loaded:", len(sales_fact))

# Close connection
conn.close()

print("\n--- LOAD COMPLETE: retail_dw.db CREATED ---")
print("Database saved at:", DB_PATH)



--- LOADING INTO SQLITE DATA WAREHOUSE ---
CustomerDim loaded: 0
TimeDim loaded: 0
SalesFact loaded: 0

--- LOAD COMPLETE: retail_dw.db CREATED ---
Database saved at: C:\Users\USER\OneDrive\Desktop\Data Warehousing\DSA204 final Exam\retail_dw.db


In [18]:
import pandas as pd
import numpy as np
import sqlite3
import traceback

def run_full_etl(input_csv_path, output_db_path):

    print("\n================= STARTING ETL PROCESS =================")

    try:
        # =====================================================
        # EXTRACT
        # =====================================================
        print("\n[1] EXTRACTING DATA...")

        df = pd.read_csv(input_csv_path, low_memory=False)

        print(f"Rows extracted: {len(df)}")

        # Make safe copy
        df = df.copy()

        # Convert Date
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"])

        print(f"Rows after fixing Date: {len(df)}")


        # =====================================================
        # TRANSFORM
        # =====================================================
        print("\n[2] TRANSFORMING DATA...")

        # ---- New Columns ----
        df["TotalSales"] = df["Qty"] * df["Amount"]
        df["OrderMonth"] = df["Date"].dt.to_period("M").astype(str)
        df["OrderYear"] = df["Date"].dt.year

        df["B2B_Flag"] = df["B2B"].apply(
            lambda x: 1 if str(x).strip().lower() == "yes" else 0
        )

        df["ShippingRegion"] = df["ship-country"].apply(
            lambda x: "Local (Kenya)" if str(x).strip().lower() == "kenya" else "International"
        )

        df["ValueCategory"] = pd.cut(
            df["TotalSales"],
            bins=[0, 1000, 5000, np.inf],
            labels=["Low", "Medium", "High"]
        )

        # ---- Remove Outliers ----
        before_clean = len(df)
        df = df[(df["Qty"] >= 0) & (df["Amount"] > 0)]
        after_clean = len(df)

        print(f"Outliers removed: {before_clean - after_clean}")

        # ---- Filter last 3 months ----
        max_date = df["Date"].max()
        three_months_ago = max_date - pd.DateOffset(months=3)

        df_last_3m = df[df["Date"].between(three_months_ago, max_date)]

        print(f"Max date in dataset: {max_date}")
        print(f"Filtering from last 3 months: {three_months_ago}")
        print(f"Rows in last 3 months: {len(df_last_3m)}")


        # =====================================================
        # DIMENSIONS
        # =====================================================
        print("\n[3] BUILDING DIMENSIONS...")

        # ---- Customer Dimension ----
        customer_dim = (
            df_last_3m.groupby(["ship-postal-code", "ship-city", "ship-country"])
            .agg(
                TotalOrders=("Order ID", "count"),
                TotalQuantity=("Qty", "sum"),
                TotalSpent=("TotalSales", "sum"),
            )
            .reset_index()
        )

        customer_dim.insert(0, "CustomerID", range(1, len(customer_dim) + 1))

        print(f"CustomerDim rows: {len(customer_dim)}")

        # ---- Time Dimension ----
        time_dim = (
            df_last_3m[["Date"]]
            .drop_duplicates()
            .sort_values("Date")
            .reset_index(drop=True)
        )

        time_dim.insert(0, "TimeID", range(1, len(time_dim) + 1))
        time_dim["Day"] = time_dim["Date"].dt.day
        time_dim["Month"] = time_dim["Date"].dt.month
        time_dim["Year"] = time_dim["Date"].dt.year
        time_dim["Quarter"] = time_dim["Date"].dt.quarter

        print(f"TimeDim rows: {len(time_dim)}")


        # =====================================================
        # FACT TABLE
        # =====================================================
        print("\n[4] BUILDING FACT TABLE...")

        # Merge dimensions
        fact = df_last_3m.merge(
            customer_dim,
            on=["ship-postal-code", "ship-city", "ship-country"],
            how="left"
        ).merge(
            time_dim,
            on="Date",
            how="left"
        )

        # Select fact columns
        sales_fact = fact[[
            "Order ID", "SKU", "Qty", "Amount", "TotalSales",
            "CustomerID", "TimeID", "ShippingRegion", "ValueCategory", "B2B_Flag"
        ]].copy()

        print(f"SalesFact rows: {len(sales_fact)}")


        # =====================================================
        # LOAD
        # =====================================================
        print("\n[5] LOADING INTO SQLITE DATABASE...")

        conn = sqlite3.connect(output_db_path)

        customer_dim.to_sql("CustomerDim", conn, if_exists="replace", index=False)
        time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)
        sales_fact.to_sql("SalesFact", conn, if_exists="replace", index=False)

        conn.close()

        print("\n⭐⭐⭐ ETL COMPLETED SUCCESSFULLY ⭐⭐⭐")

    except Exception as e:
        print("\n❌ ETL FAILED DUE TO AN ERROR:")
        print(str(e))
        traceback.print_exc()





In [19]:
INPUT_CSV = r"C:\Users\USER\OneDrive\Desktop\Data Warehousing\DSA204 final Exam\Data\Amazon Sale Report.csv"
OUTPUT_DB = r"C:\Users\USER\OneDrive\Desktop\Data Warehousing\DSA204 final Exam\retail_dw.db"

run_full_etl(INPUT_CSV, OUTPUT_DB)



================= STARTING ETL PROCESS =================

[1] EXTRACTING DATA...
Rows extracted: 128975
Rows after fixing Date: 128975

[2] TRANSFORMING DATA...


C:\Users\USER\AppData\Local\Temp\ipykernel_22244\689340709.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


Outliers removed: 10138
Max date in dataset: 2022-06-29 00:00:00
Filtering from last 3 months: 2022-03-29 00:00:00
Rows in last 3 months: 118837

[3] BUILDING DIMENSIONS...
CustomerDim rows: 17530
TimeDim rows: 91

[4] BUILDING FACT TABLE...
SalesFact rows: 118837

[5] LOADING INTO SQLITE DATABASE...

⭐⭐⭐ ETL COMPLETED SUCCESSFULLY ⭐⭐⭐


C:\Users\USER\AppData\Local\Temp\ipykernel_22244\271561111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")


(Timestamp('2022-03-31 00:00:00'), Timestamp('2022-06-29 00:00:00'))